In [47]:
import pandas as pd 
import string
import numpy as np

# Задача 1

In [48]:
df = pd.read_csv('names.txt' )
result_names = 0
names = df.columns.sort_values().tolist()
liter_dict = {letter : num + 1 for num, letter in enumerate(list(string.ascii_uppercase))}
for num_name, name in enumerate(names):
    result_word = 0
    for i in name:        
        result_word +=liter_dict.get(i)    
    result_names += result_word * (num_name+1)
print(result_names)

871853874


# Задача 2

In [78]:
df = pd.read_csv('hits.txt' , sep='\t', names=['host','ip', 'page'])
df.ip.value_counts()[:6].index

Index(['154.157.157.156', '82.146.232.163', '226.247.119.128', '194.78.107.33',
       '21.143.243.182', '226.181.188.147'],
      dtype='object')

In [79]:
dict(zip(df.ip.value_counts().index[:6],df.ip.value_counts().values[:6]))

{'154.157.157.156': 1531,
 '82.146.232.163': 1505,
 '226.247.119.128': 1494,
 '194.78.107.33': 1494,
 '21.143.243.182': 1479,
 '226.181.188.147': 1471}

# Задача 3

In [51]:
les = pd.read_csv('lessons.txt', sep='|')
part = pd.read_csv('participants.txt', sep='|')
qual = pd.read_csv('quality.txt', sep='|')
users = pd.read_csv('users.txt', sep='|')

# удаляем неопределённые поля

In [52]:
users.rename(columns={users.columns[0]:'id', users.columns[1]:'role'}, inplace=True)
part.rename(columns={part.columns[0]:'event_id', part.columns[1]:'user_id'},inplace=True)
qual.rename(columns={qual.columns[0]:'lesson_id', qual.columns[1]:'tech_quality'},inplace=True)
les.rename(columns = {les.columns[0]:'id', les.columns[1]:'event_id', les.columns[2]:'subject', les.columns[3]:'scheduled_time'}\
           ,inplace=True)

In [53]:
users = users[users.iloc[:,-1].isnull() == False]
part = part[part.iloc[:,-1].isnull() == False]
qual = qual[qual.iloc[:,-1].isnull() == False]
les = les[les.iloc[:,-1].isnull() == False]

In [54]:
part['event_id'] = part['event_id'].str.replace(' ','')
part['user_id'] = part['user_id'].str.replace(' ','')
part['gr'] = 1
part = part.groupby(by=['user_id','event_id'], as_index=False, observed=True).mean()[['event_id','user_id']]

In [56]:
users['gr'] = 1
users = users.groupby(by=['id', 'role' ], as_index=False, observed=True).mean()[['id', 'role' ]]

In [57]:
part['event_id'] = part['event_id'].apply(lambda x: int(x) if x.isalnum() else 0)

# Выделяем id_event количество которых более 1 так как нам не нужны в рассмотрении уроки прошедшие без учеников.

In [58]:
lesson = part['event_id'].value_counts()[part['event_id'].value_counts().values > 1].index.tolist()

# Оставляем в рассмотрении уроки в которых принимали участие и учитель и ученик, а так же переводим время из UTC в московское

In [59]:
les['id'] = les['id'].str.replace(' ','')
les = les[les['event_id'].isin(lesson)]
les['scheduled_time'] = pd.to_datetime(les['scheduled_time'], utc=True).dt.tz_convert('Europe/Moscow').dt.date

# Преобразуем данные tech_quality в числовые и для каждого уолка выводим сумму оценок и количество оценок (табл quality)

In [61]:
qual['tech_quality'] = qual['tech_quality'].str.replace(' ','')
qual['tech_quality'] = qual['tech_quality'].apply(lambda x: int(x) if x.isalnum() else 0)

In [62]:
qual['lesson_id'] = qual['lesson_id'].str.replace(' ','')
quality = qual.groupby(by='lesson_id', as_index=False).agg(['sum', 'count'])
quality.columns = quality.columns.droplevel(0)
quality = quality.reset_index()

# К таблие part присоединяем идентификацию участников

In [66]:
users['id'] = users['id'].str.replace(' ','')
part = part.join(users.set_index('id'), on='user_id')

# Выделяем занятия только по физике и добавляем данные о сумме оценок за урок и количество оценок к таблицу les

In [68]:
les_phis = les[les['subject'] == np.sort(les['subject'].unique())[-1]]
les_phis = les_phis.join(quality.set_index('lesson_id'), on='id').fillna(0)

In [69]:
les_phis = les_phis.join(part[part['role'] == ' tutor'].set_index('event_id'), on='event_id')
les_phis.loc[(les_phis['count'] != 0)&(les_phis['sum'] == 0), 'sum'] = 0

In [76]:
les_phis = les_phis[['scheduled_time', 'user_id', 'sum', 'count']].groupby(by=['scheduled_time', 'user_id'], as_index=False).sum()
les_phis = les_phis[les_phis['sum'] > 0]
les_phis['mean'] = round(les_phis['sum']/les_phis['count'],2)
les_phis[['scheduled_time', 'user_id','mean']].groupby(by='scheduled_time',as_index=False).min()

,scheduled_time,user_id,mean
0,2020-01-11,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,3.75
1,2020-01-12,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,4.40
2,2020-01-13,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,5.00
3,2020-01-14,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,3.33
4,2020-01-15,603b8641-c6f6-4d89-ac89-88e50d45aa0d,5.00
5,2020-01-16,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,4.00
6,2020-01-17,30a19496-bdaf-461c-abbc-2709ae520201,4.50
7,2020-01-18,1c39e78f-6ded-4b2e-83d6-036ca34ecfdc,4.25
8,2020-01-19,43efce48-94b2-4412-857f-223d45969008,4.44
9,2020-01-20,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,4.50
